In [3]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/exploratory/species_individual_peaks")

library(GSA)
library(dplyr)
library(scales)
library(qvalue)
library(ggplot2)
library(data.table)

source("/Users/rebecca/sudmant/analyses/myotis/code/fisher_test.R")

myo_meta <- read.csv("/Users/rebecca/sudmant/analyses/myotis/data/myotis_meta.csv")

## Get differential peak status between species in orthologous genes present in ALL species

peak_ortho_files <- list.files(path = "results/data", 
                               pattern = "genes_5000.*orthologs",
                               full.names = TRUE)

In [24]:
## Plot # of genes (with orthologs) per species:

peak_gene_list <- lapply(seq_along(myo_meta$Abbr), function(i) {
  file_paths <- peak_ortho_files[grep(myo_meta$Abbr[i], peak_ortho_files)]
  indv_list <- lapply(seq_along(file_paths), function(j) {
    indv_peak_ortho <- fread(file_paths[j], data.table = FALSE)
    indv_id <- sapply(strsplit(file_paths[j], "_"), "[", 2)
    df <- indv_peak_ortho %>%
      dplyr::filter(Myotis_Alias != "") %>%
      dplyr::group_by(Myotis_Alias) %>%
      dplyr::reframe(Peak = paste(unique(Peak), collapse = ", ")) %>%
      dplyr::mutate(
        Peak = ifelse(grepl(",", Peak), TRUE, Peak),
        Species = myo_meta$Field_Name[i],
        Individual = indv_id
      )
    return(df)
  })
  return(do.call(rbind, indv_list))
})
df <- do.call(rbind, peak_gene_list)

df <- df %>%
  dplyr::group_by(Species, Individual, Peak) %>%
  dplyr::reframe(
    No.Genes = n()
  )
  
x_order <- df %>%
  dplyr::filter(Peak == TRUE) %>%
  dplyr::group_by(Species) %>%
  dplyr::reframe(n = sum(No.Genes)) %>%
  dplyr::arrange(desc(n))
  
df$Species <- factor(df$Species, levels = x_order$Species)

ggplot(df, aes(x = Individual, y = No.Genes, fill = Peak)) +
  geom_bar(stat = "identity") +
  theme_minimal() +
  theme(axis.title.x = element_blank(),
        axis.text.x = element_text(angle = 45, hjust = 1),
        axis.title.y = element_text(margin = margin(r = 15)),
        panel.grid = element_blank(),
        plot.margin = unit(c(1, 1, 1, 1), "cm")) +
  labs(title = "Orthologous genes") +
  xlab("Individual") + ylab("# genes") +
  scale_y_continuous(labels = comma) +
  facet_wrap(. ~ Species, scales = "free_x")

In [4]:
## Get genes shared between all species:

gene_list <- lapply(seq_along(peak_ortho_files), function(i) {
  peak_ortho <- fread(peak_ortho_files[i], data.table = FALSE)
  return(unique(peak_ortho$Myotis_Alias[peak_ortho$Myotis_Alias != ""]))
})

shared_genes <- Reduce(intersect, gene_list)
length(shared_genes)

[1] 13774

In [5]:
## Get peak status of shared genes:

gene_peak_list <- lapply(seq_along(peak_ortho_files), function(i) {
  peak_ortho <- fread(peak_ortho_files[i], data.table = FALSE)
  peak_ortho <- peak_ortho %>%
    dplyr::filter(Myotis_Alias %in% shared_genes) %>%
    dplyr::group_by(Myotis_Alias) %>%
    dplyr::reframe(Peak = ifelse(
      sum(Peak) > 0, TRUE, FALSE
    ))
  return(unique(peak_ortho$Myotis_Alias[peak_ortho$Peak == TRUE]))
})
names(gene_peak_list) <- sapply(strsplit(sapply(strsplit(peak_ortho_files, "/"), "[", 3), "_"), function(x) paste(x[1:2], collapse = " "))

spec_peak_status <- as.data.frame.matrix(table(stack(gene_peak_list)))

In [7]:
## Plot species peak status:

library(RColorBrewer)
library(ComplexHeatmap)

cor_mat <- cor(spec_peak_status)
diag(cor_mat) <- NA

n_genes <- nrow(spec_peak_status)
plot_title <- paste("Peak status over", comma(n_genes), "shared orthologous genes")

meta_df <- data.frame(
  Species = sapply(strsplit(colnames(cor_mat), " "), "[", 1),
  row.names = colnames(cor_mat)
)
                                                       
colors <- brewer.pal(n_distinct(meta_df$Species), "Paired")
names(colors) <- unique(meta_df$Species)
meta_cols <- list(Species = colors)
                            
col_anno <- HeatmapAnnotation(df = meta_df, col = meta_cols)
row_anno <- HeatmapAnnotation(df = meta_df, col = meta_cols, which = "row", 
                              show_legend = FALSE, show_annotation_name = FALSE) 

In [ ]:
## Plot peak status heatmap

# Note: this takes a few minuts to plot

pdf(file = paste0("results/figures/gene_peak_status_heatmap.pdf"), width = 9, height = 9)

draw(
  ComplexHeatmap::Heatmap(name = "Peak status", 
                          column_title = plot_title,
                          matrix = spec_peak_status, 
                          clustering_method_rows = "average",
                          clustering_method_columns = "average",
                          show_row_names = FALSE,
                          top_annotation = col_anno,
                          column_names_rot = 45),
  padding = unit(c(1, 1, 1, 1), "cm")
)

dev.off()

In [ ]:
## Plot peak status correlation heatmap

pdf(file = paste0("results/figures/gene_peak_status_correlation_heatmap.pdf"), width = 12, height = 9)

draw(
  ComplexHeatmap::Heatmap(name = "Peak status correlation", 
                          column_title = plot_title,
                          matrix = cor_mat, 
                          clustering_method_rows = "average",
                          clustering_method_columns = "average",
                          top_annotation = col_anno,
                          left_annotation = row_anno),
  padding = unit(c(1, 1, 1, 1), "cm")
)

dev.off()

In [44]:
## Cluster genes based on peak status:

gene_dist <- dist(spec_peak_status, method = "euclidean")
gene_status_dendro <- hclust(gene_dist, method = "average")

sil_scores <- lapply(2:10, function(k) {
  clusters <- cutree(gene_status_dendro, k = k)
  ss <- cluster::silhouette(clusters, gene_dist)
  return(mean(ss[,3]))
})

max_k <- which.max(unlist(sil_scores)) + 1
clusters <- cutree(gene_status_dendro, k = max_k)

In [43]:
## Visualize clusters:

# Note: this takes a few minuts to plot

## Column annotations (species):
meta_col_df <- data.frame(
  Species = sapply(strsplit(colnames(cor_mat), " "), "[", 1),
  row.names = colnames(spec_peak_status)
)
spec_colors <- brewer.pal(n_distinct(meta_col_df$Species), "Paired")
names(spec_colors) <- unique(meta_col_df$Species)
meta_col_cols <- list(Species = spec_colors)
col_anno <- HeatmapAnnotation(df = meta_col_df, col = meta_col_cols)

## Row annotations (gene clusters):
meta_row_df <- data.frame(Cluster = clusters, 
                          row.names = rownames(spec_peak_status))
clust_colors <- brewer.pal(n_distinct(meta_row_df$Cluster), "Set2")[1:n_distinct(meta_row_df$Cluster)]
names(clust_colors) <- unique(meta_row_df$Cluster)
meta_row_cols <- list(Cluster = clust_colors)
row_anno <- HeatmapAnnotation(
  df = meta_row_df, col = meta_row_cols, which = "row",
  show_annotation_name = TRUE
) 

pdf(file = paste0("results/figures/gene_peak_status_gene_clusters_heatmap.pdf"), width = 12, height = 9)

draw(
  ComplexHeatmap::Heatmap(name = "Peak status", 
                          column_title = plot_title,
                          matrix = spec_peak_status, 
                          clustering_method_rows = "average",
                          clustering_method_columns = "average",
                          show_row_names = FALSE,
                          top_annotation = col_anno,
                          right_annotation = row_anno),
   padding = unit(c(1, 1, 1, 1), "cm")
)

dev.off()

pdf 
  2

In [51]:
## What human homologs are associated with the genes in each cluster?

ortho_table <- fread("/Users/rebecca/sudmant/analyses/myotis/data/ortholog_mapping.csv", data.table = FALSE)
yum_col <- grep("^yum", colnames(ortho_table))
ortho_table[,yum_col] <- gsub("SCAF", "SUPER", ortho_table[,yum_col]) 
colnames(ortho_table)[1:3] <- c("ENSEMBL", "SYMBOL", "Myotis_Alias")

clust_alias_list <- tapply(names(clusters), clusters, "[")
clust_symbol_list <- lapply(clust_alias_list, function(cluster) {
  genes <- unique(ortho_table$SYMBOL[ortho_table$Myotis_Alias %in% cluster])  
  return(genes[!grepl("^ENS", genes)])
})

all_genes <- ortho_table$SYMBOL[ortho_table$Myotis_Alias %in% shared_genes]
all_genes <- all_genes[!grepl("^ENS", all_genes)]

In [ ]:
## Run enrichment analysis on genes in each cluster:

## Load in Broad gene sets:
gsc <- GSA.read.gmt("/Users/rebecca/sudmant/analyses/myotis/data/genesets/msigdb_v2023.2.Hs_GMTs/msigdb.v2023.2.Hs.symbols.gmt")
sets <- gsc$genesets
names(sets) <- gsc$geneset.names

In [61]:
cluster_enrichments <- lapply(clust_symbol_list, function(clust_genes) {
  clust_enrich <- sort(unlist(lapply(sets, fisher_test,
                                     mod = clust_genes,
                                     all = all_genes)))
  return(data.frame(Set = names(clust_enrich), 
                    Pval = clust_enrich,
                    Qval = qvalue(clust_enrich)$qvalues,
                    row.names = NULL))
})

In [62]:
lapply(cluster_enrichments, head, 10)

,Set,Pval,Qval
,<chr>,<dbl>,<dbl>
1,HP_ABNORMAL_HEART_MORPHOLOGY,8.651672e-25,1.514291e-20
2,HP_ABNORMALITY_OF_THE_HAND,1.410192e-24,1.514291e-20
3,HP_ABNORMALITY_OF_LIMB_BONE,1.891373e-24,1.514291e-20
4,HP_DECREASED_HEAD_CIRCUMFERENCE,5.104438e-24,3.065076e-20
5,HP_ABNORMAL_FACIAL_SKELETON_MORPHOLOGY,8.167602e-24,3.923538e-20
6,ZSCAN30_TARGET_GENES,2.555576e-23,1.023036e-19
7,HP_ABNORMAL_NASAL_MORPHOLOGY,4.497310e-23,1.543150e-19
8,HP_APLASIA_HYPOPLASIA_INVOLVING_THE_SKELETON,6.880185e-23,2.065682e-19
9,HP_ABNORMALITY_OF_SPEECH_OR_VOCALIZATION,9.356796e-23,2.497111e-19
